In [4]:
!pip install -q gradio transformers

In [5]:
# import gradio as gr
# import transformers
# import torch.nn.functional as F
# import numpy as np

# # def generate(model_name="Salesforce/codegen-350M-mono", text="World"):
# #     model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
# #     tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
# #     input_ids = tokenizer.encode(text, return_tensors='pt')
# #     output = model.generate(input_ids, max_length=100, do_sample=True)
# #     return tokenizer.decode(output[0])

# def get_token_likelihoods(model_name="Salesforce/codegen-350M-mono", text="World"):
#     # get likelihoods for each token
#     model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
#     tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
#     # start_token = self.enc(self.enc.bos_token, return_tensors='pt').data['input_ids'][0]
#     input_ids = tokenizer.encode(text, return_tensors='pt')
#     out = model(input_ids)
#     probs = F.softmax(out.logits, dim=-1).squeeze()
    
#     output = []
#     for tok, logits in zip(input_ids.squeeze(), probs):
#         output.append((
#             tokenizer.decode(tok),
#             str(round(logits[tok].item() * 100, 4)) + "%",
#             # tokenizer.decode(np.argmax(logits.detach()))
#         ))

#     return output

# code_example = \
# """<|endoftext|># Handshake successful, kill previous client if there is any.
# with current_client_pid.get_lock():
#     old_pid = current_client_pid.value
#     if old_pid != 0:
#         print(f"Booting previous client (pid={old_pid})")
#         os.kill(old_pid, signal.SIGKILL)
#         current_client_pid.value = os.getpid()"""

# # demo = gr.Interface(
# #     fn=get_token_likelihoods,
# #     title="Heatmap Demo",
# #     inputs = [
# #         gr.Textbox(
# #             label="Model name",
# #             lines=1,
# #             value="Salesforce/codegen-350M-mono",
# #         ),
# #         gr.Textbox(
# #             label="Text",
# #             lines=8,
# #             value=code_example,
# #         ),
# #     ],
# #     # outputs = gr.HighlightedText(
# #     #     label="Probs",
# #     # ).style(color_map={"+": "red", "-": "green"}),
# #     outputs = gr.Markdown()
# # )

# def welcome(code_example):
#     # return f"Welcome to Gradio, {name}!"
#     return get_token_likelihoods(text=code_example)

# with gr.Blocks() as demo:
#     inp = gr.Textbox(label='Code', placeholder=code_example, value=code_example)
#     out = gr.Textbox(label='Heatmap')
#     inp.change(welcome, inp, out)

# if __name__ == "__main__":
#     demo.launch()

In [199]:
# import gradio as gr
# # import transformers
# # import torch.nn.functional as F
# import numpy as np

# from transformers import (AutoModelForCausalLM, AutoTokenizer)
# import torch


# class AbstractLanguageChecker:
#     """
#     Abstract Class that defines the Backend API of GLTR.

#     To extend the GLTR interface, you need to inherit this and
#     fill in the defined functions.
#     """

#     def __init__(self):
#         """
#         In the subclass, you need to load all necessary components
#         for the other functions.
#         Typically, this will comprise a tokenizer and a model.
#         """
#         self.device = torch.device(
#             "cuda" if torch.cuda.is_available() else "cpu")

#     def check_probabilities(self, in_text, topk=40):
#         """
#         Function that GLTR interacts with to check the probabilities of words

#         Params:
#         - in_text: str -- The text that you want to check
#         - topk: int -- Your desired truncation of the head of the distribution

#         Output:
#         - payload: dict -- The wrapper for results in this function, described below

#         Payload values
#         ==============
#         bpe_strings: list of str -- Each individual token in the text
#         real_topk: list of tuples -- (ranking, prob) of each token
#         pred_topk: list of list of tuple -- (word, prob) for all topk
#         """
#         raise NotImplementedError

#     def postprocess(self, token):
#         """
#         clean up the tokens from any special chars and encode
#         leading space by UTF-8 code '\u0120', linebreak with UTF-8 code 266 '\u010A'
#         :param token:  str -- raw token text
#         :return: str -- cleaned and re-encoded token text
#         """
#         raise NotImplementedError


# def top_k_logits(logits, k):
#     """
#     Filters logits to only the top k choices
#     from https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_gpt2.py
#     """
#     if k == 0:
#         return logits
#     values, _ = torch.topk(logits, k)
#     min_values = values[:, -1]
#     return torch.where(logits < min_values,
#                        torch.ones_like(logits, dtype=logits.dtype) * -1e10,
#                        logits)


# class LM(AbstractLanguageChecker):
#     def __init__(self, model_name_or_path="Salesforce/codegen-350M-multi"):
#         super(LM, self).__init__()
#         self.enc = AutoTokenizer.from_pretrained(model_name_or_path)
#         self.model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
#         self.model.to(self.device)
#         self.model.eval()
#         self.start_token = self.enc(self.enc.bos_token, return_tensors='pt').data['input_ids'][0]
#         print(f"Loaded model: {model_name_or_path}")

#     def check_probabilities(self, in_text, topk=40):
#         # Process input
#         token_ids = self.enc(in_text, return_tensors='pt').data['input_ids'][0]
#         token_ids = torch.concat([self.start_token, token_ids])
#         # Forward through the model
#         output = self.model(token_ids.to(self.device))
#         all_logits = output.logits[:-1].detach().squeeze()
#         # construct target and pred
#         # yhat = torch.softmax(logits[0, :-1], dim=-1)
#         all_probs = torch.softmax(all_logits, dim=1)

#         y = token_ids[1:]
#         # Sort the predictions for each timestep
#         sorted_preds = torch.argsort(all_probs, dim=1, descending=True).cpu()
#         # [(pos, prob), ...]
#         real_topk_pos = list(
#             [int(np.where(sorted_preds[i] == y[i].item())[0][0])
#              for i in range(y.shape[0])])
#         real_topk_probs = all_probs[np.arange(
#             0, y.shape[0], 1), y].data.cpu().numpy().tolist()
#         real_topk_probs = list(map(lambda x: round(x, 5), real_topk_probs))

#         real_topk = list(zip(real_topk_pos, real_topk_probs))
#         # [str, str, ...]
#         bpe_strings = self.enc.convert_ids_to_tokens(token_ids[:])

#         bpe_strings = [self.postprocess(s) for s in bpe_strings]

#         topk_prob_values, topk_prob_inds = torch.topk(all_probs, k=topk, dim=1)

#         pred_topk = [list(zip(self.enc.convert_ids_to_tokens(topk_prob_inds[i]),
#                               topk_prob_values[i].data.cpu().numpy().tolist()
#                               )) for i in range(y.shape[0])]
#         pred_topk = [[(self.postprocess(t[0]), t[1]) for t in pred] for pred in pred_topk]


#         # pred_topk = []
#         payload = {'bpe_strings': bpe_strings,
#                    'real_topk': real_topk,
#                    'pred_topk': pred_topk}
#         if torch.cuda.is_available():
#             torch.cuda.empty_cache()

#         return payload
    
#     def postprocess(self, token):
#         return token
    
# lm = LM()

Loaded model: Salesforce/codegen-350M-multi


In [274]:
in_text = code_example = \
"""# Handshake successful, kill previous client if there is any.
with current_client_pid.get_lock():
    old_pid = current_client_pid.value
    if old_pid != 0:
        print(f"Booting previous client (pid={old_pid})")
        os.kill(old_pid, signal.SIGKILL)
        current_client_pid.value = os.getpid()"""

# lm = LM()

# def get(code_example):
#     payload = lm.check_probabilities(code_example)
#     # bpe_strings = payload['bpe_strings'][1:]
#     # topk = payload['real_topk']
#     return payload['pred_topk']

# with gr.Blocks() as demo:
#     inp = gr.Textbox(label='Code example', placeholder=code_example, value=code_example)
#     out = gr.Textbox(label='Heatmap')
#     inp.change(get, inp, out)

# if __name__ == "__main__":
#     demo.launch()

In [270]:

# # Process input
# token_ids = lm.enc(code_example, return_tensors='pt').data['input_ids'][0]
# token_ids = torch.concat([lm.start_token, token_ids])
# token_ids
# # Forward through the model
# output = lm.model(token_ids.to(lm.device))
# output
# all_logits = output.logits[:-1].detach().squeeze()
# # construct target and pred
# # yhat = torch.softmax(logits[0, :-1], dim=-1)
# all_probs = torch.softmax(all_logits, dim=1)

# y = token_ids[1:]
# # Sort the predictions for each timestep
# sorted_preds = torch.argsort(all_probs, dim=1, descending=True).cpu()
# # [(pos, prob), ...]
# real_topk_pos = list(
#     [int(np.where(sorted_preds[i] == y[i].item())[0][0])
#         for i in range(y.shape[0])])
# real_topk_probs = all_probs[np.arange(
#     0, y.shape[0], 1), y].data.cpu().numpy().tolist()
# real_topk_probs = list(map(lambda x: round(x, 5), real_topk_probs))

# real_topk = list(zip(real_topk_pos, real_topk_probs))
# # [str, str, ...]
# bpe_strings = lm.enc.convert_ids_to_tokens(token_ids[:])

# bpe_strings = [lm.postprocess(s) for s in bpe_strings]

# topk_prob_values, topk_prob_inds = torch.topk(all_probs, k=topk, dim=1)

# pred_topk = [list(zip(lm.enc.convert_ids_to_tokens(topk_prob_inds[i]),
#                         topk_prob_values[i].data.cpu().numpy().tolist()
#                         )) for i in range(y.shape[0])]
# pred_topk = [[(lm.postprocess(t[0]), t[1]) for t in pred] for pred in pred_topk]


# # pred_topk = []
# payload = {'bpe_strings': bpe_strings,
#             'real_topk': real_topk,
#             'pred_topk': pred_topk}
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()

In [271]:
# token_ids = lm.enc(in_text, return_tensors='pt').data['input_ids'][0]
# # token_ids = torch.concat([lm.start_token, token_ids])
# # Forward through the model
# output = lm.model(token_ids.to(lm.device))
# logits = output.logits.data
# print('logits.shape: ', logits.shape)

logits.shape:  torch.Size([97, 51200])


In [ ]:
import transformers
import torch
from torch import Tensor
from typing import Tuple, List


def get_tokens_and_logits(in_text: str, model_name: str, device: str = None) -> Tuple[List[str], Tensor]:
    model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    print('Model: ', model.__class__.__name__)
    print('Adding bos token...')
    in_text = f'{tokenizer.bos_token}{in_text}'
    print('Input:')
    print(in_text)
    print('------')
    inputs: Tensor = tokenizer(in_text, return_tensors='pt').data['input_ids'].squeeze()
    print("Input token ids' shape: ", inputs.shape)
    if not device:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    print('Device: ', device)
    outputs: Tensor = model(inputs.to(torch.device(device)))
    # NOTE: The last logit vector is used for predicting the next token, which is not part of the input. Therefore, we exclude it.
    print('Discarding last logit vector...')
    logits: Tensor = outputs.logits.data[:-1]
    print('logits.shape: ', logits.shape)
    tokens: List[str] = tokenizer.decode(inputs[1:])
    return tokens, logits

def get_scores(logits: Tensor) -> Tensor:
    ret: Tensor = torch.distributions.Categorical(logits=logits).entropy()
    print('scores.shape: ', ret.shape)
    return ret

# def get_topk(logits: Tensor, topk: int = 5):
#     topk_prob_values, topk_prob_inds = torch.topk(logits, k=topk, dim=1)
#     return topk_prob_values, topk_prob_inds

tokens, logits = get_tokens_and_logits(code_example, model_name='Salesforce/codegen-350M-mono')
scores = get_scores(logits)

Model:  CodeGenForCausalLM
Adding bos token...
Input:
<|endoftext|># Handshake successful, kill previous client if there is any.
with current_client_pid.get_lock():
    old_pid = current_client_pid.value
    if old_pid != 0:
        print(f"Booting previous client (pid={old_pid})")
        os.kill(old_pid, signal.SIGKILL)
        current_client_pid.value = os.getpid()
------
Input token ids' shape:  torch.Size([98])
Device:  cpu
Discarding last logit vector...
logits.shape:  torch.Size([97, 51200])


In [174]:
logits -= logits.min(dim=-1, keepdim=True).values
logits /= logits.max(dim=-1, keepdim=True).values
logits

tensor([[0.8421, 0.8407, 0.8675,  ..., 0.0477, 0.0475, 0.0475],
        [0.6350, 0.5575, 0.5599,  ..., 0.0682, 0.0678, 0.0679],
        [0.8095, 0.7691, 0.7231,  ..., 0.0457, 0.0455, 0.0456],
        ...,
        [0.4795, 0.5214, 0.6179,  ..., 0.0676, 0.0675, 0.0676],
        [0.6694, 0.6995, 0.7520,  ..., 0.0511, 0.0512, 0.0512],
        [0.7615, 0.7607, 0.8356,  ..., 0.0477, 0.0478, 0.0478]])

In [340]:
import gradio as gr
import matplotlib as mpl
import numpy as np
from typing import List


def get_colors(scores: Tensor) -> List[str]:
    cmap = mpl.colormaps['YlOrBr']
    rgbas: np.ndarray = cmap(scores)
    return np.apply_along_axis(mpl.colors.rgb2hex, -1, rgbas)

def get_markdown_heatmap(in_text: str):
        tokens, logits = get_tokens_and_logits(in_text, model_name='Salesforce/codegen-350M-mono')
        scores = get_scores(logits)
        colors = get_colors(scores)
        assert len(tokens) == len(colors)
        ret = ''.join([f'<span style="background-color: {c}">{t}</span>' for t, c in zip(tokens, colors)])
        return f'```{ret}'

# with gr.Blocks() as demo:
#     gr.Markdown(
#     """
#     # Demo
#     """)
#     inp = gr.Textbox(label='Code example', placeholder=code_example, value=code_example)
#     tokens, logits = get_tokens_and_logits(inp, model_name='Salesforce/codegen-350M-mono')
#     scores = get_scores(logits)
#     colors = get_colors(scores)
#     out = gr.Markdown(label="Heatmap")
#     btn = gr.Button(value="Run")
#     btn.click(get_markdown_heatmap, inputs=[tokens, colors], outputs=out)
demo = gr.Interface(
    fn=get_markdown_heatmap,
    inputs=gr.Textbox(label='Code example', placeholder=code_example, value=code_example),
    outputs=gr.Markdown()
)


if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7888

To create a public link, set `share=True` in `launch()`.


Model:  CodeGenForCausalLM
Adding bos token...
Input:
<|endoftext|># Handshake successful, kill previous client if there is any.
with current_client_pid.get_lock():
    old_pid = current_client_pid.value
    if old_pid != 0:
        print(f"Booting previous client (pid={old_pid})")
        os.kill(old_pid, signal.SIGKILL)
        current_client_pid.value = os.getpid()
------
Input token ids' shape:  torch.Size([98])
Device:  cpu
Discarding last logit vector...
logits.shape:  torch.Size([97, 51200])
scores.shape:  torch.Size([97])


Traceback (most recent call last):
  File "/Users/nadavt/opt/anaconda3/envs/detecting-fake-text/lib/python3.8/site-packages/gradio/routes.py", line 384, in run_predict
    output = await app.get_blocks().process_api(
  File "/Users/nadavt/opt/anaconda3/envs/detecting-fake-text/lib/python3.8/site-packages/gradio/blocks.py", line 1032, in process_api
    result = await self.call_function(
  File "/Users/nadavt/opt/anaconda3/envs/detecting-fake-text/lib/python3.8/site-packages/gradio/blocks.py", line 844, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/nadavt/opt/anaconda3/envs/detecting-fake-text/lib/python3.8/site-packages/anyio/to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "/Users/nadavt/opt/anaconda3/envs/detecting-fake-text/lib/python3.8/site-packages/anyio/_backends/_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "/Users/na

In [314]:
get_colors(scores)

array(['#662506', '#662506', '#662506', '#662506', '#662506', '#662506',
       '#662506', '#662506', '#662506', '#662506', '#662506', '#662506',
       '#662506', '#662506', '#662506', '#662506', '#fec24d', '#662506',
       '#662506', '#662506', '#662506', '#662506', '#fecc61', '#662506',
       '#f17b1a', '#ffeea9', '#ae3e03', '#662506', '#ee7617', '#662506',
       '#fecc61', '#db5d0b', '#ffffe5', '#fffbcf', '#fffddc', '#fffddd',
       '#fed26d', '#fea736', '#fea030', '#662506', '#662506', '#d55607',
       '#ffffe5', '#ffffe5', '#662506', '#662506', '#fec857', '#fff7bc',
       '#fffcd6', '#662506', '#662506', '#de610c', '#882f05', '#662506',
       '#662506', '#662506', '#dd5f0c', '#662506', '#662506', '#662506',
       '#ffeea8', '#ffffe5', '#ffffe5', '#ee7416', '#662506', '#fff7bd',
       '#fee390', '#662506', '#fffcd4', '#feb340', '#fff1ae', '#fecc61',
       '#ffffe5', '#ffffe5', '#fffddd', '#8c3004', '#fffcd4', '#fffacb',
       '#ffeea9', '#662506', '#ffffe5', '#fffcd8', 